In [1]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from sklearn.metrics import (confusion_matrix, precision_score, auc,
                             roc_curve, recall_score, classification_report, f1_score)
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import scipy.stats as stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
from sklearn.cluster import KMeans
from sklearn_som.som import SOM
from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn import tree
import glob
import sys
np.set_printoptions(threshold=sys.maxsize)

# Dados NSL-KDD

In [2]:
kddtrain = pd.read_csv(r"C:\Users\luang\OneDrive\Documentos\Mestrado\Códigos Python\Autoencoder-based Network Anomaly Detection\KDDTrain.csv")
kddtest = pd.read_csv(r"C:\Users\luang\OneDrive\Documentos\Mestrado\Códigos Python\Autoencoder-based Network Anomaly Detection\KDDTest.csv")

In [21]:
kddtrain.head()


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [17]:
x = pd.concat((kddtrain,kddtest), axis=0)
x = pd.get_dummies(x)
data_class = x['class_anomaly']
data = x.drop(['class_anomaly', 'class_normal'], axis = 1) 

In [18]:
data.shape[1]

116

In [19]:
data = data.to_numpy()
data_class = data_class.to_numpy()

In [20]:
noTrain = np.count_nonzero(data_class == 0)
print(noTrain)
anTrain = np.count_nonzero(data_class == 1)
print(anTrain)

9711
12833


In [11]:
num_folds = 10

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)
for train, test in kfold.split(data, data_class):
    print(train.shape[0])
    print(test.shape[0])
    noTrain = np.count_nonzero(data_class[train] == 0)
    print("Normal:",noTrain)
    anTrain = np.count_nonzero(data_class[train] == 1)
    print("Anomalo:",anTrain)

133665
14852
Normal: 69348
Anomalo: 64317
133665
14852
Normal: 69348
Anomalo: 64317
133665
14852
Normal: 69348
Anomalo: 64317
133665
14852
Normal: 69348
Anomalo: 64317
133665
14852
Normal: 69349
Anomalo: 64316
133665
14852
Normal: 69349
Anomalo: 64316
133665
14852
Normal: 69349
Anomalo: 64316
133666
14851
Normal: 69349
Anomalo: 64317
133666
14851
Normal: 69349
Anomalo: 64317
133666
14851
Normal: 69349
Anomalo: 64317


# Abordagem Autoencoder-Algoritmos

In [5]:
epochs = 100
batch_size = 256
optimizer = Adam(learning_rate=0.0001)
activation = "relu"
neighbors = 11

num_folds = 10

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

fold_no = 1
count = 0

#Métricas ErroRecons
matrix_ErroRecons = np.zeros((num_folds,2,2))
accuracy_ErroRecons = np.zeros(num_folds)
recall_ErroRecons = np.zeros(num_folds)
precision_ErroRecons = np.zeros(num_folds)
f1_ErroRecons = np.zeros(num_folds)
Z = np.arange(0.000,4.095,0.005)
bestZ = np.zeros(num_folds)
bestTeta = np.zeros(num_folds)
bestmean = np.zeros(num_folds)
bestdesv = np.zeros(num_folds)
bestErrorNumber = np.zeros(num_folds)
ErrorNumberFolds = np.zeros((num_folds,1,Z.shape[0]-1))

#Métricas KNN
matrix_knn = np.zeros((num_folds,2,2))
accuracy_knn = np.zeros(num_folds)
recall_knn = np.zeros(num_folds)
precision_knn = np.zeros(num_folds)
f1_knn = np.zeros(num_folds)

#Métricas Decision Tree
matrix_dt = np.zeros((num_folds,2,2))
accuracy_dt = np.zeros(num_folds)
recall_dt = np.zeros(num_folds)
precision_dt = np.zeros(num_folds)
f1_dt = np.zeros(num_folds)

#Métricas SVM
matrix_svm = np.zeros((num_folds,2,2))
accuracy_svm = np.zeros(num_folds)
recall_svm = np.zeros(num_folds)
precision_svm = np.zeros(num_folds)
f1_svm = np.zeros(num_folds)

for train, test in kfold.split(data, data_class):
    
    #####################################################
    ####Pré-Processamento
    
    #Separação dos Folds de Treinamento e Teste
    dataTrain = data[train]
    dataTrain_class = data_class[train]
    modelTest = data[test]
    modelTest_class = data_class[test]
    
    #Conta Quantidade de dados
    Train = dataTrain.shape[1]
    noTrain = np.count_nonzero(dataTrain_class == 0)
    anTrain = np.count_nonzero(dataTrain_class == 1)

    #Separa dados normais e anômalos
    normalTrain = dataTrain[np.where(dataTrain_class == 0)]
    anomalyTrain = dataTrain[np.where(dataTrain_class == 1)]
    
    #Divide dados para treinamento
    porcen = 0.50
    j = int(noTrain * porcen)
    
    #Índices escolhidos aleatoriamente
    numbers_knn_normal = np.array(random.sample(range(0, noTrain),j))
    numbers_knn_anomaly = np.array(random.sample(range(0,anTrain),j))
    numbers_autoencoder_normal = np.array(list(set(np.arange(0, noTrain)) - set(numbers_knn_normal)))
    
    #Dados KNN
    knnTrain_normal = normalTrain[numbers_knn_normal]
    knnTrain_anomaly = anomalyTrain[numbers_knn_anomaly]
    knnTrain = np.concatenate((knnTrain_normal, knnTrain_anomaly), axis=0)
    knnTrain_class = np.concatenate((np.zeros(j), np.ones(j)))
    
    #Dados Autoencoder
    autoencoderTrain = normalTrain[numbers_autoencoder_normal]
    autoencoderTrain_class = np.zeros(autoencoderTrain.shape[0])
    
    #Normalização
    scaler = MinMaxScaler()
    scaler.fit(knnTrain)
    autoencoderTrain = scaler.transform(autoencoderTrain)
    knnTrain = scaler.transform(knnTrain)
    modelTest = scaler.transform(modelTest)
    
    #####################################################
    ####Autoencoder
    #Modela as camadas do Autoencoder
    var = autoencoderTrain.shape[1]
    input_vector = keras.Input(shape=(var,))
    en1 = layers.Dense(32, activation=activation)(input_vector)
    en2 = layers.Dense(16, activation=activation)(en1)
    de1 = layers.Dense(32, activation=activation)(en2)
    de2 = layers.Dense(var, activation=activation)(de1)

    #Gera o modelo
    autoencoder = keras.Model(input_vector, de2)

    #Compila o modelo
    autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")

    #Treina o modelo
    history = autoencoder.fit(autoencoderTrain, autoencoderTrain, epochs = epochs, batch_size = batch_size, shuffle = True)
    
    #####################################################
    ####Erro de Reconstrução
    #Teste
    pred_ErroRecons = autoencoder.predict(modelTest)
    
    #Extração do erro
    mse_ErroRecons = np.mean(np.power(modelTest - pred_ErroRecons, 2), axis=1)
    error_df_ErroRecons = pd.DataFrame({'reconstruction_error': mse_ErroRecons,
                                        'true_class': modelTest_class})

    media = np.mean(error_df_ErroRecons['reconstruction_error'])
    desv = np.std(error_df_ErroRecons['reconstruction_error'])

    #Determinação do limiar ótimo

    ErrosLimiar = np.zeros(Z.shape[0]-1)
    melhorTeta = 0
    melhorZ = 0
    prediMatrix = np.zeros((Z.shape[0]-1, 1, modelTest.shape[0]))
    
    for i in range(0, Z.shape[0]-1):
        #Obtenção do Limiar
        teta = media + Z[i]*desv
    
        #Predição
        predi_ErroRecons = np.ones(modelTest.shape[0])
        error_ErroRecons = error_df_ErroRecons.to_numpy()
        predi_ErroRecons[np.where(error_ErroRecons[:,0] <= teta)] = 0
        prediMatrix[i] = predi_ErroRecons 
        
        #Contagem da quantidade de erros obtidos para cada Z
        numberErros = 0
        for j in range(0, predi_ErroRecons.shape[0]):
            if error_ErroRecons[j,1] != predi_ErroRecons[j]:
                numberErros = numberErros + 1
        ErrosLimiar[i] = numberErros
        
    predi_ErroRecons = prediMatrix[np.argpartition(ErrosLimiar,1)[0]]
            
    #Métricas
    accuracy_ErroRecons[count] = accuracy_score(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    recall_ErroRecons[count] = recall_score(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    precision_ErroRecons[count] = precision_score(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    f1_ErroRecons[count] = f1_score(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    matrix_ErroRecons[count] = confusion_matrix(error_ErroRecons[:,1], predi_ErroRecons[:].reshape(-1,1))
    bestZ[count] = Z[np.argpartition(ErrosLimiar,1)[0]]
    bestTeta[count] = media + Z[np.argpartition(ErrosLimiar,1)[0]]*desv
    bestmean[count] = media
    bestdesv[count] = desv
    bestErrorNumber[count] = ErrosLimiar[np.argpartition(ErrosLimiar,1)[0]]
    ErrorNumberFolds[count] = ErrosLimiar
    
    #####################################################
    ####KNN
    ###Treinamento KNN
    pred_knn = autoencoder.predict(knnTrain)
    mse_knn = np.mean(np.power(knnTrain - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                 'true_class': knnTrain_class})
    error_knn = error_df_knn.to_numpy()
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(error_knn[:,0].reshape(-1,1), error_knn[:,1])
    
    #Teste do Modelo
    pred_knn = autoencoder.predict(modelTest)
    mse_knn = np.mean(np.power(modelTest - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                'true_class': modelTest_class})
    error_knn = error_df_knn.to_numpy()
    predict_knn = neigh.predict(error_knn[:,0].reshape(-1,1))
    
    #Métricas
    accuracy_knn[count] = accuracy_score(error_knn[:,1], predict_knn[:])
    recall_knn[count] = recall_score(error_knn[:,1], predict_knn[:])
    precision_knn[count] = precision_score(error_knn[:,1], predict_knn[:])
    f1_knn[count] = f1_score(error_knn[:,1], predict_knn[:])
    matrix_knn[count] = confusion_matrix(error_knn[:,1], predict_knn[:])
    
   
    
    #####################################################
    ####Decision Tree
    ###Treinamento Decision Tree 
    pred_dt = autoencoder.predict(knnTrain)
    mse_dt = np.mean(np.power(knnTrain - pred_dt, 2), axis=1)
    error_df_dt = pd.DataFrame({'reconstruction_error': mse_dt,
                                     'true_class': knnTrain_class})
    error_dt = error_df_dt.to_numpy()
    dt = tree.DecisionTreeClassifier()
    dt.fit(error_dt[:,0].reshape(-1,1), error_dt[:,1])

    #Teste
    pred_dt = autoencoder.predict(modelTest)
    mse_dt = np.mean(np.power(modelTest - pred_dt, 2), axis=1)
    error_df_dt = pd.DataFrame({'reconstruction_error': mse_dt,
                                     'true_class': modelTest_class})
    error_dt = error_df_dt.to_numpy()
    predict_dt = dt.predict(error_dt[:,0].reshape(-1,1))

    #Métricas
    accuracy_dt[count] = accuracy_score(error_dt[:,1], predict_dt[:])
    recall_dt[count] = recall_score(error_dt[:,1], predict_dt[:])
    precision_dt[count] = precision_score(error_dt[:,1], predict_dt[:])
    f1_dt[count] = f1_score(error_dt[:,1], predict_dt[:])
    matrix_dt[count] = confusion_matrix(error_dt[:,1], predict_dt[:])
    
   

    #####################################################
    ####SVM
    #Treinamento SVM
    pred_svm = autoencoder.predict(knnTrain)
    mse_svm = np.mean(np.power(knnTrain - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                 'true_class': knnTrain_class})
    error_svm = error_df_svm.to_numpy()
    clf = svm.SVC(kernel='rbf')
    clf.fit(error_svm[:,0].reshape(-1,1), error_svm[:,1])

    #Teste
    pred_svm = autoencoder.predict(modelTest)
    mse_svm = np.mean(np.power(modelTest - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                 'true_class': modelTest_class})
    error_svm = error_df_svm.to_numpy()
    predict_svm = clf.predict(error_svm[:,0].reshape(-1,1))
    
    #Métricas
    accuracy_svm[count] = accuracy_score(error_svm[:,1], predict_svm[:])
    recall_svm[count] = recall_score(error_svm[:,1], predict_svm[:])
    precision_svm[count] = precision_score(error_svm[:,1], predict_svm[:])
    f1_svm[count] = f1_score(error_svm[:,1], predict_svm[:])
    matrix_svm[count] = confusion_matrix(error_svm[:,1], predict_svm[:])
    
    
    
      
    count = count + 1
    fold_no = fold_no + 1
    


Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
136/136 [==============================] - 1s 3ms/step - loss: 0.0551
Epoch 2/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0447
Epoch 3/100
136/136 [==============================] - 1s 4ms/step - loss: 0.0311A: 0s - loss: 0.03
Epoch 4/100
136/136 [==============================] - 0s 4ms/step - loss: 0.0261
Epoch 5/100
136/136 [===========

136/136 [==============================] - 0s 3ms/step - loss: 0.0074
Epoch 64/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0074
Epoch 65/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0074
Epoch 66/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0074
Epoch 67/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0074
Epoch 68/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0074
Epoch 69/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0074
Epoch 70/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0074
Epoch 71/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0074
Epoch 72/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0074
Epoch 73/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0074
Epoch 74/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0074
Epoch 75/100
136/

136/136 [==============================] - 0s 3ms/step - loss: 0.0181
Epoch 16/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0181
Epoch 17/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0180
Epoch 18/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0180
Epoch 19/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0180
Epoch 20/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0180
Epoch 21/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0180
Epoch 22/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0179
Epoch 23/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0144
Epoch 24/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0116
Epoch 25/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0115
Epoch 26/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0114A: 0s - loss:
Epoc

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
136/136 [==============================] - 1s 2ms/step - loss: 0.0275
Epoch 2/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0104
Epoch 3/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0079
Epoch 4/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 5/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0056
Epoch 6/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0052
Epoch 7/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0049
Epoch 8/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0047
Epoch 9/100
136/136 [========================

Epoch 70/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 71/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 72/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 73/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 74/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 75/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 76/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 77/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 78/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 79/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 80/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 81/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0071
Epoc

136/136 [==============================] - 0s 3ms/step - loss: 0.0154
Epoch 23/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0153
Epoch 24/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0153
Epoch 25/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0153
Epoch 26/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0153
Epoch 27/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0153
Epoch 28/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0153
Epoch 29/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0153
Epoch 30/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0152
Epoch 31/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0152
Epoch 32/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0152
Epoch 33/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0152
Epoch 34/100
136/

136/136 [==============================] - 1s 2ms/step - loss: 0.0399
Epoch 2/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0262
Epoch 3/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0201
Epoch 4/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0187
Epoch 5/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0182
Epoch 6/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0170
Epoch 7/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0104
Epoch 8/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0100
Epoch 9/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0097
Epoch 10/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0096
Epoch 11/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0094
Epoch 12/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0093
Epoch 13/100
136/136 [===

136/136 [==============================] - 0s 3ms/step - loss: 0.0033
Epoch 76/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 77/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 78/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 79/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 80/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 81/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 82/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 83/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 84/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 85/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 86/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 87/100
136/

In [6]:
print("Autoencoder - Limiar Referência:\n",np.mean(matrix_ErroRecons, axis=0), "\nAcurácia: ", np.mean(accuracy_ErroRecons),' +/- ',np.std(accuracy_ErroRecons), "\nRecall: ", np.mean(recall_ErroRecons),' +/- ',np.std(recall_ErroRecons), "\nPrecision: ", np.mean(precision_ErroRecons),' +/- ',np.std(precision_ErroRecons), "\nF1: ", np.mean(f1_ErroRecons),' +/- ',np.std(f1_ErroRecons))
for i in range(0,fold_no-1):
    print("\nFold ",i+1,": " , bestTeta[i], "=", bestmean[i], "+", bestZ[i], "*", bestdesv[i], "\n\nNúmero de Erros: ", bestErrorNumber[i])

Autoencoder - Limiar Referência:
 [[7369.7  335.7]
 [1326.5 5819.8]] 
Acurácia:  0.88808017572815  +/-  0.023474395009095303 
Recall:  0.8143791149132383  +/-  0.04266982311105274 
Precision:  0.9464743955677047  +/-  0.03060818822748716 
F1:  0.8746419530529158  +/-  0.027371113175074333

Fold  1 :  0.01660062316075063 = 0.01660062316075063 + 0.0 * 0.018853169960511673 

Número de Erros:  1429.0

Fold  2 :  0.017636196765374302 = 0.017636196765374302 + 0.0 * 0.01406899230389916 

Número de Erros:  1617.0

Fold  3 :  0.014564876646302833 = 0.014564876646302833 + 0.0 * 0.010111435008653588 

Número de Erros:  2234.0

Fold  4 :  0.018168802463172883 = 0.017836986516696428 + 0.035 * 0.009480455613612965 

Número de Erros:  1884.0

Fold  5 :  0.012044189263176327 = 0.011875866787267918 + 0.015 * 0.01122149839389386 

Número de Erros:  1116.0

Fold  6 :  0.01570527731128143 = 0.01570527731128143 + 0.0 * 0.012401236272478756 

Número de Erros:  1664.0

Fold  7 :  0.017536031734078646 = 0.017

In [7]:
print("Autoencoder - KNN:\n", np.mean(matrix_knn, axis=0), "\nAcurácia: ", np.mean(accuracy_knn), ' +/- ', np.std(accuracy_knn), "\nRecall: ", np.mean(recall_knn), ' +/- ', np.std(recall_knn), "\nPrecision: ", np.mean(precision_knn), ' +/- ', np.std(precision_knn), "\nF1: ", np.mean(f1_knn), ' +/- ', np.std(f1_knn))

print("\nAutoencoder - SVM:\n", np.mean(matrix_svm, axis=0), "\nAcurácia: ", np.mean(accuracy_svm), ' +/- ', np.std(accuracy_svm), "\nRecall: ", np.mean(recall_svm), ' +/- ', np.std(recall_svm), "\nPrecision: ", np.mean(precision_svm), ' +/- ', np.std(precision_svm), "\nF1: ", np.mean(f1_svm), ' +/- ', np.std(f1_svm))

print("\nAutoencoder - Decision Tree:\n", np.mean(matrix_dt, axis=0), "\nAcurácia: ", np.mean(accuracy_dt), ' +/- ', np.std(accuracy_dt), "\nRecall: ", np.mean(recall_dt), ' +/- ', np.std(recall_dt), "\nPrecision: ", np.mean(precision_dt), ' +/- ', np.std(precision_dt), "\nF1: ", np.mean(f1_dt), ' +/- ', np.std(f1_dt))

Autoencoder - KNN:
 [[7183.4  522. ]
 [ 642.8 6503.5]] 
Acurácia:  0.9215713496501735  +/-  0.01227403996613153 
Recall:  0.9100514989858919  +/-  0.015888533156893418 
Precision:  0.9257289596253313  +/-  0.012668093871829086 
F1:  0.9177814427313754  +/-  0.013017695898368417

Autoencoder - SVM:
 [[7278.8  426.6]
 [1131.7 6014.6]] 
Acurácia:  0.8950758248335295  +/-  0.019576319535779953 
Recall:  0.8416380396934848  +/-  0.03865548219485491 
Precision:  0.9347300740269249  +/-  0.026088568522749166 
F1:  0.8850087967873236  +/-  0.022876965520127353

Autoencoder - Decision Tree:
 [[7188.6  516.8]
 [ 814.2 6332.1]] 
Acurácia:  0.9103807217770676  +/-  0.015145223933448955 
Recall:  0.8860668866913054  +/-  0.017470157009622032 
Precision:  0.9245333979870856  +/-  0.015705904148126226 
F1:  0.9048735769388244  +/-  0.016185213721226176


In [8]:
Result_GridSearchLimiar = np.array(["Theta: " , bestTeta,"",
                                "Média: ", bestmean,"",
                                "Valor Z: ", bestZ,"",
                                "Desvio Padrao: ", bestdesv,"",
                                "Número de Erros: ", bestErrorNumber,"",
                                "Erros por Fold: ", ErrorNumberFolds])

np.savetxt('Result_GridSearchLimiarRef_Error.txt', Result_GridSearchLimiar, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [9]:
Result = np.array(["Referência", "Acurácia", accuracy_ErroRecons, np.mean(accuracy_ErroRecons), np.std(accuracy_ErroRecons),"",
                                "Recall", recall_ErroRecons, np.mean(recall_ErroRecons), np.std(recall_ErroRecons), "",
                                "Precision", precision_ErroRecons, np.mean(precision_ErroRecons), np.std(precision_ErroRecons), "",
                                "F1", f1_ErroRecons, np.mean(f1_ErroRecons), np.std(f1_ErroRecons), "-----------------","-----------------",
                   "KNN", "Acurácia", accuracy_knn, np.mean(accuracy_knn), np.std(accuracy_knn),"",
                                "Recall", recall_knn, np.mean(recall_knn), np.std(recall_knn), "",
                                "Precision", precision_knn, np.mean(precision_knn), np.std(precision_knn), "",
                                "F1", f1_knn, np.mean(f1_knn), np.std(f1_knn), "-----------------","-----------------",
                  "Decision Tree",  "Acurácia", accuracy_dt, np.mean(accuracy_dt), np.std(accuracy_dt),"",
                                "Recall", recall_dt, np.mean(recall_dt), np.std(recall_dt), "",
                                "Precision", precision_dt, np.mean(precision_dt), np.std(precision_dt), "",
                                "F1", f1_dt, np.mean(f1_dt), np.std(f1_dt), "-----------------","-----------------",
                  "SVM",  "Acurácia", accuracy_svm, np.mean(accuracy_svm), np.std(accuracy_svm),"",
                                "Recall", recall_svm, np.mean(recall_svm), np.std(recall_svm), "",
                                "Precision", precision_svm, np.mean(precision_svm), np.std(precision_svm), "",
                                "F1", f1_svm, np.mean(f1_svm), np.std(f1_svm), "","",])

np.savetxt('Result_Error.txt', Result, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


In [10]:
Result_Matrix = np.array(["Referência", matrix_ErroRecons, np.mean(matrix_ErroRecons, axis=0), np.std(matrix_ErroRecons, axis=0), "-----------------","-----------------",
                          "KNN", matrix_knn, np.mean(matrix_knn, axis=0), np.std(matrix_knn, axis=0), "-----------------","-----------------",
                          "Decision Tree", matrix_dt, np.mean(matrix_dt, axis=0), np.std(matrix_dt, axis=0), "-----------------","-----------------",
                          "SVM", matrix_svm, np.mean(matrix_svm, axis=0), np.std(matrix_svm, axis=0)])

np.savetxt('Result_Error_Matrix.txt', Result_Matrix, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.
